In [1]:
import wandb
import os
api = wandb.Api()

sweep = api.sweep("ravikumarchavva-org/T20I-Cricket-Win-Prediction/xpxta7kj")
runs = sorted(sweep.runs,
  key=lambda run: run.summary.get("val_accuracy", 0), reverse=True)
val_acc = runs[0].summary.get("val_accuracy", 0)
print(f"Best run {runs[0].name} with {val_acc}% validation accuracy")

config = runs[0].config
config

Best run bumbling-sweep-2 with 80.2588996763754% validation accuracy


{'lr': 0.001,
 'dropout': 0.6440042563772401,
 'save_dir': 'D:\\github\\Cricket-Prediction\\ml_modeling\\4_hptuning',
 'gpu_count': 1,
 'batch_size': 8,
 'num_epochs': 50,
 'num_layers': 2,
 'hidden_size': 128,
 'enable_plots': False,
 'weight_decay': 4.112676091367997e-05,
 'learning_rate': 0.0009586290162506808,
 'torch_version': '2.4.1',
 'cuda_available': True,
 'python_version': '3.9.20 (main, Oct  3 2024, 07:38:01) [MSC v.1929 64 bit (AMD64)]'}

In [2]:
wandb.init(project="T20I-Cricket-Win-Prediction", job_type="inference")

# Use the artifact name as logged during training
artifact = wandb.use_artifact('T20I-Cricket-Win-Prediction/best_model:latest', type='model')  # Replace 'your_entity' with your WandB username or team name
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ravikumarchavva (ravikumarchavva-org). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [7]:
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

from model_utils import CricketDataset, collate_fn_with_padding, extract_data, augment_match_data
from torch.utils.data import DataLoader
import pickle

# Load the Datasets
train_dataset = pickle.load(open(os.path.join(os.getcwd(), '..', "data", "pytorch_data", 'train_dataset.pkl'), 'rb'))
val_dataset = pickle.load(open(os.path.join(os.getcwd(), '..', "data", "pytorch_data", 'val_dataset.pkl'), 'rb'))
test_dataset = pickle.load(open(os.path.join(os.getcwd(), '..', "data", "pytorch_data", 'test_dataset.pkl'), 'rb'))

# Step 1: Extract Data from Dataset
train_team_data, train_player_data, train_ball_data, train_labels = extract_data(train_dataset)
val_team_data, val_player_data, val_ball_data, val_labels = extract_data(val_dataset)
test_team_data, test_player_data, test_ball_data, test_labels = extract_data(test_dataset)

# Step 2: Augment Data
train_ball_data, train_team_data, train_player_data, train_labels = augment_match_data(train_ball_data, train_team_data, train_player_data, train_labels)
val_ball_data, val_team_data, val_player_data, val_labels = augment_match_data(val_ball_data, val_team_data, val_player_data, val_labels)
test_ball_data, test_team_data, test_player_data, test_labels = augment_match_data(test_ball_data, test_team_data, test_player_data, test_labels)

# Step 3: Convert augmented data to PyTorch Dataset
train_dataset = CricketDataset(train_team_data, train_player_data, train_ball_data, train_labels)
val_dataset = CricketDataset(val_team_data, val_player_data, val_ball_data, val_labels)
test_dataset = CricketDataset(test_team_data, test_player_data, test_ball_data, test_labels) 

# Step 4: Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_with_padding)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_with_padding)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn_with_padding)

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from model_utils import  EncoderDecoderModel

model = EncoderDecoderModel(
    team_input_size=train_team_data[0].shape[0],
    player_input_channels=1,  # Assuming player data is 2D and needs a channel dimension
    ball_input_size=train_ball_data[0].shape[1],
    hidden_size=128,
    num_layers=1,
    num_classes=1,
    dropout=0.5631325927251238
).to(device)  # Move model to device
    

model.load_state_dict(torch.load(os.path.join(artifact_dir, 'best_model.pth',), map_location=device, weights_only=True))

c:\Users\chavv\anaconda\envs\huggingface-torch\lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5631325927251238 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RuntimeError: Error(s) in loading state_dict for EncoderDecoderModel:
	Unexpected key(s) in state_dict: "player_encoder.fc1.weight", "player_encoder.fc1.bias", "ball_encoder.rnn.weight_ih_l1", "ball_encoder.rnn.weight_hh_l1", "ball_encoder.rnn.bias_ih_l1", "ball_encoder.rnn.bias_hh_l1". 
	size mismatch for team_encoder.fc1.weight: copying a param with shape torch.Size([256, 13]) from checkpoint, the shape in current model is torch.Size([128, 13]).
	size mismatch for team_encoder.fc1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for team_encoder.batch_norm1.weight: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for team_encoder.batch_norm1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for team_encoder.batch_norm1.running_mean: copying a par

In [84]:
model.load_state_dict(torch.load(os.path.join(artifact_dir, 'best_model.pth'), map_location=device))

C:\Users\chavv\AppData\Local\Temp\ipykernel_37924\43199998.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(artifact_dir, 'b

<All keys matched successfully>

In [79]:
from model_utils import  EncoderDecoderModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EncoderDecoderModel(
    team_input_size=train_team_data[0].shape[0],
    player_input_channels=1,  # Assuming player data is 2D and needs a channel dimension
    ball_input_size=train_ball_data[0].shape[1],
    hidden_size=config['hidden_size'],
    num_layers=config['num_layers'],
    num_classes=1,
    dropout=config['dropout']
).to(device)  # Move model to device

save_dir = os.path.join(os.getcwd(), 'model_checkpoints')
os.makedirs(save_dir, exist_ok=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=20, factor=0.1)

# Step 4: Train Model
best_val_loss = float('inf')
patience = 10
trigger_times = 0
num_epochs = 100

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    for team, player, ball, labels in tqdm(train_dataloader):
        team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)  # Move data to device
        labels = labels.float()
        outputs = model(team, player, ball)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = (outputs.data > 0.5).float()
        total += labels.size(0)
        running_corrects += (predicted == labels).sum().item()
    avg_loss = running_loss / len(train_dataloader)
    train_acc = 100 * running_corrects / total

    train_losses.append(avg_loss)
    train_accuracies.append(train_acc)

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for team, player, ball, labels in val_dataloader:
            team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)  # Move data to device
            labels = labels.float()
            outputs = model(team, player, ball)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predicted = (outputs.data > 0.5).float()
            val_total += labels.size(0)
            val_corrects += (predicted == labels).sum().item()
    avg_val_loss = val_loss / len(val_dataloader)
    val_acc = 100 * val_corrects / val_total

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {train_acc:.2f}%, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    # Log metrics to Weights & Biases
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc
    })

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_times = 0
        # Save the best model locally
        torch.save(model.state_dict(), os.path.join(save_dir, 'best_model.pth'))
        # Create a Weights & Biases Artifact
        model_artifact = wandb.Artifact('best_model', type='model')
        model_artifact.add_file(os.path.join(save_dir, 'best_model.pth'))
        # Log the Artifact
        wandb.log_artifact(model_artifact)
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break

100%|██████████| 78/78 [00:04<00:00, 18.07it/s]


Epoch [1/100], Loss: 0.7046, Acc: 49.64%, Val Loss: 0.7059, Val Acc: 58.25%


100%|██████████| 78/78 [00:03<00:00, 19.61it/s]


Epoch [2/100], Loss: 0.6790, Acc: 49.76%, Val Loss: 0.6805, Val Acc: 58.25%


100%|██████████| 78/78 [00:03<00:00, 19.75it/s]


...

Epoch [23/100], Loss: 0.6191, Acc: 76.40%, Val Loss: 0.6960, Val Acc: 71.84%


100%|██████████| 78/78 [00:04<00:00, 19.33it/s]


Epoch [24/100], Loss: 0.6171, Acc: 77.06%, Val Loss: 0.6660, Val Acc: 74.27%


100%|██████████| 78/78 [00:04<00:00, 19.36it/s]


Epoch [25/100], Loss: 0.6222, Acc: 77.10%, Val Loss: 0.6893, Val Acc: 72.17%
Early stopping!


In [80]:
model.eval()
all_labels = []
all_predictions = []
all_probs = []

# Define window sizes
window_sizes = [20, 25, 30, 35, 40, 45]
stage_metrics = {}
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc
with torch.no_grad():
    correct = 0
    total = 0
    for team, player, ball, labels in test_dataloader:
        team, player, ball, labels = team.to(device), player.to(device), ball.to(device), labels.to(device)  # Move data to device
        team = team.float()
        player = player.float()
        ball = ball.float()
        labels = labels.float()
        
        outputs = model(team, player, ball)
        probs = outputs.squeeze().cpu().numpy()
        if probs.ndim == 0:
            probs = probs.reshape(1)  # Convert scalar to 1D array
        predicted = (outputs.data > 0.5).float()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        all_probs.extend(probs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Test Accuracy: {:.2f} %'.format(100 * correct / total))


Test Accuracy: 78.53 %
